# Model Formulation

## Sets
Let $T$ be a set of time periods (months), whre $t_0 \in T$ is the first month and $t_e \in T$ the last month.

Let $P$ be a set of products and $M$ be a set of machines.

## Parameters
- For each product $p \in P$ and each type of machine $m \in M$, we are given the time $f_{p,m}$ (in hours) the product $p \in P$ needs to be manufactured on the machine $m \in M$.
- For each month $t \in T$ and each product $p \in P$, we are given the upper limit on sales of $l_{t,p}$ for that product in that month. 
- For each product $p \in P$, we are given the profit $k_p$.
- For each month $t \in T$ and each machione $m \in M$, we are given the number of available machines $q_{t,m}$.
- Each machine can work $g$ hours a month.
- There can be $z$ products of each type stored in each month and storing cost $r$ per product per month occur.

## Variables
For each month $t \in T$ and each product $p \in P$, we introduce continuous non-negative variables $b_{t,p}, u_{t,p}, s_{t,p}$.
- $b_{t,p}$ describes how much we produce of the product $p \in P$ in the month $t \in T$.
- $u_{t,p}$ describes how much we sell of the product $p \in P$ in the month $t \in T$.
- $s_{t,p}$ describes how much we store of the product $p \in P$ in the month $t \in T$.

## Objective function
The Objective is to maximize the profit of the company. It consists of the profit for each product minus cost for storing the unsold products. This can be stated as 

$ \max \sum_{t \in T} \sum_{p \in P} (k_p u_{t,p} - r s_{t,p})$

## Constrains
The balance constraints ensure that the amount is in the storage in the last month and the amount that get manufactured equals the amount that is sold and held for each product in the current month. This makes sure that all products in the model are manufactured in some month. The initial storage is empty. 

$ s_{t-1,p} + b_{t,p} = u_{t,p} + s_{t,p} \qquad \forall t \in T \ t_0, \forall p \in P $

$ b_{t_0,p} = u_{t_0,p} + s_{t_0,p} \qquad \qquad \forall p \in P $

The endstore constrains force that at the end of the last month the storage contains the specified amount of each product (a full storage).

$ s_{t_e,p} = z \qquad \forall p \in P $

The store capacity constraints restrict the amount of each product, which can be stored in each month. At most $z=50$ units of each product be stored in each month.

$s_{t,p} \leq z \qquad \forall p \in P, \forall t \in T$

The capacity constraints ensure that per month the time that all products needs on a certain kind of machine is lower or equal than the available hours for that machine is that month multiplied by the number of available machine in that month. Each product needs some machine hour on different machines. Each machine is down in one or more month due to maintenance, so the number of available machines varies per month. There can be multiplied machines per machine type.

$ \sum_{p \in P} f_{p,m} b_{t,p} \leq g q_{t,m} \qquad \forall t \in T, \forall m \in M $

# Python Implementation

In [1]:
from gurobipy import *

In [2]:
products = ["Prod1", "Prod2", "Prod3", "Prod4", "Prod5", "Prod6", "Prod7"]
machines = ["grinder", "vertDrill", "horiDrill", "borer", "planer"]
time_periods = ["January", "February", "March", "April", "May", "June"]

In [3]:
profit_contribution = [10, 6, 8, 4, 11, 9, 3]

time_table = {
    "grinder": {    "Prod1": 0.5, "Prod2": 0.7, "Prod5": 0.3,
                    "Prod6": 0.2, "Prod7": 0.5 },
    "vertDrill": {  "Prod1": 0.1, "Prod2": 0.2, "Prod4": 0.3,
                    "Prod6": 0.6 },
    "horiDrill": {  "Prod1": 0.2, "Prod3": 0.8, "Prod7": 0.6 },
    "borer": {      "Prod1": 0.05,"Prod2": 0.03,"Prod4": 0.07,
                    "Prod5": 0.1, "Prod7": 0.08 },
    "planer": {     "Prod3": 0.01,"Prod5": 0.05,"Prod7": 0.05 }
}

In [4]:
# number of machines down
down = {("January","grinder"): 1, ("February", "horiDrill"): 2, ("March", "borer"): 1,
        ("April", "vertDrill"): 1, ("May", "grinder"): 1, ("May", "vertDrill"): 1,
        ("June", "planer"): 1, ("June", "horiDrill"): 1}

qMachine = [4, 2, 3, 1, 1] # number of each machine available

# market limitation of sells
upper = [
        [500, 1000,  300,  300,  800,  200,  100],
        [600,  500,  200,    0,  400,  300,  150],
        [300,  600,    0,    0,  500,  400,  100],
        [200,  300,  400,  500,  200,    0,  100],
        [  0,  100,  500,  100, 1000,  300,    0],
        [500,  500,  100,  300, 1100,  500,   60]]

storeCost = 0.5
storeCapacity = 100
endStock = 50
hoursPerMonth = 2*8*24

In [5]:
model = Model('Factory Planning I')

In [6]:
# 1st step: define decision variables
manu = {} # quantity manufactured
held = {} # quantity stored
sell = {} # quantity sold

for index1, time_period in enumerate(time_periods):
    for index2, product in enumerate(products):
        manu[time_period, product] = model.addVar(vtype=GRB.CONTINUOUS,
                                                  name="Manu_{}_{}".format(product, time_period))
        held[time_period, product] = model.addVar(vtype=GRB.CONTINUOUS, ub=storeCapacity,
                                                  name="Held_{}_{}".format(product, time_period))
        sell[time_period, product] = model.addVar(vtype=GRB.CONTINUOUS, ub=upper[index1][index2],
                                                  name="Sell_{}_{}".format(product, time_period))

In [8]:
# 2nd step: integerate variables
model.update()

In [9]:
# 3rd step: define objective function
obj = quicksum(
    profit_contribution[prod_index] * sell[time_period, product] -
    storeCost * held[time_period, product]
    for time_period in time_periods
    for prod_index, product in enumerate(products)
)
model.setObjective(obj, GRB.MAXIMIZE)

In [10]:
# 4th step: define the constrains

# Balancing constrains:
for index, time_period in enumerate(time_periods):
    for product in products:
        if index == 0:
            model.addConstr(manu[time_period, product] == sell[time_period, product] + held[time_period, product],
                            "Balance_{}_{}".format(product, time_period))
        else:
            model.addConstr(held[time_periods[index-1], product] + manu[time_period, product] ==
                            sell[time_period, product] + held[time_period, product],
                            "Balance_{}_{}".format(product, time_period))
# Endstore constrains:
for product in products:
    model.addConstr(held[time_periods[len(time_periods)-1], product] == endStock, "Endbalance_{}".format(product))

# Capacity constrains:
for time_index, time_period in enumerate(time_periods):
    for mach_index, machine in enumerate(machines):
        if (time_period, machine) in down:
            model.addConstr(quicksum(time_table[machine][product] * manu[time_period, product]
                            for product in time_table[machine])
                            <= hoursPerMonth * (qMachine[mach_index] - down[time_period, machine]),
                            "Capacity_{}_{}".format(machine, time_period))
        else:
            model.addConstr(quicksum(time_table[machine][product] * manu[time_period, product]
                            for product in time_table[machine])
                            <= hoursPerMonth * qMachine[mach_index],
                            "Capacity_{}_{}".format(machine, time_period))

In [11]:
# 5th step: start optimization
model.optimize()

Optimize a model with 79 rows, 126 columns and 288 nonzeros
Coefficient statistics:
  Matrix range    [1e-02, 1e+00]
  Objective range [5e-01, 1e+01]
  Bounds range    [6e+01, 1e+03]
  RHS range       [5e+01, 2e+03]
Presolve removed 74 rows and 110 columns
Presolve time: 0.00s
Presolved: 5 rows, 16 columns, 21 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.4425000e+04   1.440000e+02   0.000000e+00      0s
       2    9.3715179e+04   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds
Optimal objective  9.371517857e+04


In [13]:
# 6th step: display solution (print the name of each variable and the solution value)
for v in model.getVars():
    print(v.getAttr("Varname"), v.getAttr("X"))

Manu_Prod1_January 500.0
Held_Prod1_January 0.0
Sell_Prod1_January 500.0
Manu_Prod2_January 888.5714285714287
Held_Prod2_January 0.0
Sell_Prod2_January 888.5714285714286
Manu_Prod3_January 382.5
Held_Prod3_January 82.5
Sell_Prod3_January 300.0
Manu_Prod4_January 300.0
Held_Prod4_January 0.0
Sell_Prod4_January 300.0
Manu_Prod5_January 800.0
Held_Prod5_January 0.0
Sell_Prod5_January 800.0
Manu_Prod6_January 200.0
Held_Prod6_January 0.0
Sell_Prod6_January 200.0
Manu_Prod7_January 0.0
Held_Prod7_January 0.0
Sell_Prod7_January 0.0
Manu_Prod1_February 700.0
Held_Prod1_February 100.0
Sell_Prod1_February 600.0
Manu_Prod2_February 600.0
Held_Prod2_February 100.0
Sell_Prod2_February 500.0
Manu_Prod3_February 117.5
Held_Prod3_February 0.0
Sell_Prod3_February 200.0
Manu_Prod4_February 0.0
Held_Prod4_February 0.0
Sell_Prod4_February 0.0
Manu_Prod5_February 500.0
Held_Prod5_February 100.0
Sell_Prod5_February 400.0
Manu_Prod6_February 300.0
Held_Prod6_February 0.0
Sell_Prod6_February 300.0
Manu_Prod7

In [1]:
!Ipython locate

C:\Users\shafieez\.ipython
